In [5]:
using System.IO;
using Newtonsoft.Json.Linq;
using Newtonsoft.Json;


class Student {
    public string Name;
    public string Group;
    public string Discipline;
    public int Mark;
}

class JsonResponse {
    public IEnumerable Response;

    public JsonResponse(IEnumerable Response) {
        this.Response = Response;
    }

    public string ToJson() {
        return JsonConvert.SerializeObject(this, Formatting.Indented);
    }

    public void Write(string path) {
        using (StreamWriter sw = new StreamWriter(path)) {
            sw.Write(this.ToJson());
        }
    }
}

List<Student> getStudents(string input_f_path) {
    List<Student> students = new List<Student>();
    using (StreamReader reader = new StreamReader(@".\input_1.json")) {
        JObject task = (JObject)JToken.ReadFrom(new JsonTextReader(reader));
        foreach(JObject s_data in task["data"]) {
            Student student = new Student() {Name = (string)s_data["name"], Group = (string)s_data["group"],
                                            Discipline = (string)s_data["discipline"], Mark = (int)s_data["mark"]};   
            students.Add(student);
        }
    }
    return students;
}

In [6]:
List<Student> students = getStudents(@".\input_1.json");

var studentsByAverageMarks = 
    from student in students
    group student by student.Name into studentsByName
    select new 
    {
        Name = studentsByName.Key,
        AverageMark = studentsByName.Average(st => st.Mark)
    };

var studentsWithMaxAverageMark = 
    from student in studentsByAverageMarks
    let maxAverageMark = studentsByAverageMarks.Max(st => st.AverageMark)
    where student.AverageMark == maxAverageMark
    select new {
        Cadet = student.Name,
        GPA = maxAverageMark
    };

new JsonResponse(studentsWithMaxAverageMark).Write(@".\output_1.json");

In [7]:
List<Student> students = getStudents(@".\input_2.json");

var averageDisciplineMarks = 
    from student in students
    group student by student.Discipline into studentsByDiscipline
    select new 
    {
        Name = studentsByDiscipline.Key,
        AverageMark = studentsByDiscipline.Average(st => st.Mark)
    };

new JsonResponse(averageDisciplineMarks).Write(@".\output_2.json");

In [8]:
List<Student> students = getStudents(@".\input_2.json");

var averageDisciplineMarks = 
    from student in students
    group student by student.Discipline into studentsByDiscipline
    select new 
    {
        Discipline = studentsByDiscipline.Key,
        Groups = (
            from student2 in studentsByDiscipline
            group student2 by student2.Group into studentsByGroup
            select new 
            {
                Group = studentsByGroup.Key,
                AverageMark = studentsByGroup.Average(st => st.Mark)
            }
        )
    };

var groupsWithMaxMarks = 
    from discipline in averageDisciplineMarks
    let average = discipline.Groups.Max(g => g.AverageMark)
    from g in discipline.Groups
    where g.AverageMark == average
    select new {
        Discipline = discipline.Discipline,
        Group = g.Group,
        GPA = g.AverageMark
    };

new JsonResponse(groupsWithMaxMarks).Write(@".\output_3.json");